In [20]:
import torch
import numpy as np
import os

In [8]:
torch.__version__

'1.13.1'

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import fftconvolve
import IPython
import pyroomacoustics as pra

from pyroomacoustics.directivities import (
    DirectivityPattern,
    DirectionVector,
    CardioidFamily,
)

In [10]:
def room_properties(absorption, size):
    if absorption=='high':
        absor = 0.7
    elif absorption=='medium':
        absor = 0.3
    elif absorption=='low':
        absor = 0.1
    else:
        raise ValueError("The absorption parameter can only take values ['low', 'medium', 'high']")
    
    if size=='large':
        size_coef = 5.0
    elif size=='medium':
        size_coef = 2.5
    elif size=='small':
        size_coef = 1.0
    else:
        raise ValueError("The absorption parameter can only take values ['low', 'medium', 'high']")
    return absor, size_coef

In [11]:
def clip(signal, high, low):
    """Clip a signal from above at high and from below at low."""
    s = signal.copy()

    s[np.where(s > high)] = high
    s[np.where(s < low)] = low

    return s

def normalize(signal, bits=None):
    """
    normalize to be in a given range. The default is to normalize the maximum
    amplitude to be one. An optional argument allows to normalize the signal
    to be within the range of a given signed integer representation of bits.
    """

    s = signal.copy()

    s /= np.abs(s).max()

    # if one wants to scale for bits allocated
    if bits is not None:
        s *= 2 ** (bits - 1) - 1
        s = clip(s, 2 ** (bits - 1) - 1, -(2 ** (bits - 1)))

    return s
    
def to_wav(filename, fs, signals, norm=False, bitdepth=np.float):
    """
    Save all the signals to wav files.

    Parameters
    ----------
    filename: str
        the name of the file
    mono: bool, optional
        if true, records only the center channel floor(M / 2) (default
        `False`)
    norm: bool, optional
        if true, normalize the signal to fit in the dynamic range (default
        `False`)
    bitdepth: int, optional
        the format of output samples [np.int8/16/32/64 or np.float
        (default)]
    """
    from scipy.io import wavfile

    signal = signals.T  # each column is a channel

    float_types = [float, np.float, np.float32, np.float64]

    if bitdepth in float_types:
        bits = None
    elif bitdepth is np.int8:
        bits = 8
    elif bitdepth is np.int16:
        bits = 16
    elif bitdepth is np.int32:
        bits = 32
    elif bitdepth is np.int64:
        bits = 64
    else:
        raise NameError("No such type.")

    if norm:
        signal = normalize(signal, bits=bits)

    signal = np.array(signal, dtype=bitdepth)

    wavfile.write(filename, fs, signal)

/tmp/ipykernel_3023/3403348603.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def to_wav(filename, fs, signals, norm=False, bitdepth=np.float):


# G1

In [12]:
import glob

In [15]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [16]:
len(a)

40

In [17]:
b = a[0:40]

In [18]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [19]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [22]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('high', 'small')
    room = pra.ShoeBox(
        p=size_coef*np.array([4, 3, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[3.5,2.5,1.])

    room.simulate()
    
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [24]:
out_path = '/nvme/zhiyong/fllearn_data/G1_ID/'
# make directory
os.makedirs(out_path)

for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G1_ID/'

        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G1_ID.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_3023/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G2

In [30]:
import glob

In [31]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [32]:
b = a

In [33]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [34]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [35]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('low', 'large')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[17.,12.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [36]:
out_path = '/nvme/zhiyong/fllearn_data/G2_ID/'
# make directory
os.makedirs(out_path)

for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G2_ID/'

        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G2_ID.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_3023/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G3

In [37]:
import glob

In [38]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [39]:
b = a

In [40]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [41]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [42]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[8.,6.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [43]:
out_path = '/nvme/zhiyong/fllearn_data/G3_ID/'
# make directory
os.makedirs(out_path)

for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G3_ID/'

        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G3_ID.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_3023/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G4

In [44]:
import glob

In [45]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [46]:
b = a

In [47]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [48]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [49]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([4.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    room.add_microphone(loc=[8.,4.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [50]:
out_path = '/nvme/zhiyong/fllearn_data/G4_ID/'
# make directory
os.makedirs(out_path)

for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G4_ID/'

        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G4_ID.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_3023/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G5

In [51]:
import glob

In [52]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [53]:
b = a

In [54]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [55]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [56]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))
    # dir_list = []
    # for i in range(6):
    #     dir_list.append(CardioidFamily(
    #     orientation=DirectionVector(azimuth=0, colatitude=90, degrees=True),
    #     pattern_enum=DirectivityPattern.OMNI,))

    # mics = pra.MicrophoneArray(R, room.fs, directivity=dir_list)
    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [57]:
out_path = '/nvme/zhiyong/fllearn_data/G5_ID/'
# make directory
os.makedirs(out_path)

for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G5_ID/'

        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G5_ID.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_3023/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G6

In [58]:
import glob

In [59]:
a = glob.glob('/nvme/zhiyong/voxceleb/vox1/test/wav/*')

In [60]:
b = a

In [61]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [62]:
b_list

['/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/AeRSD9jIdSg',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/rXRbmL7nzIo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/6uqGkiR2oTI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/UkrkPwZd77Y',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/BVFTRGHuk3M',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/Eu8Bj1gx5_4',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/vhzF7dhGJzI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/eVfmXU0RtRo',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/VIAnMw7J5tI',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10301/hX6LjjVtnGs',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/wH6ebv-K8AA',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/YCIIiIR9B60',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/7rkVfW2yw0c',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/xbGp2KSUr0E',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/89UnsytX6Y8',
 '/nvme/zhiyong/voxceleb/vox1/test/wav/id10274/L9q6XabEpEI',
 '/nvme/zhiyong/voxceleb

In [63]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([8.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))

    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [64]:
out_path = '/nvme/zhiyong/fllearn_data/G6_ID/'
# make directory
os.makedirs(out_path)

for i in b_list:
    c = glob.glob(i+'/*')
    for j in c:
        label = j.split('/')[-3]
        out_path = '/nvme/zhiyong/fllearn_data/G6_ID/'

        generate_wave(j, out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1])
        with open(out_path+'train_list_G6_ID.txt', 'a') as f:
            f.write(label+' '+out_path+label+'_'+j.split('/')[-2]+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_3023/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]
